In [1]:
# !wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [2]:
from langchain_community.llms import LlamaCpp
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)
llm.invoke("Hi! My name is Maarten. What is 1+1?")

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


''

In [3]:
# from langchain_openai import ChatOpenAI

# chat_model = ChatOpenAI(openai_api_key=OPENAIKEY)

# Prompt Template

In [4]:
# create and chain the prompt template
from langchain.prompts import PromptTemplate
template = """
<|user|>{input_prompt}<|end|>
<|assistant|>
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)
basic_chain = prompt | llm

In [5]:
# activate chain
basic_chain.invoke(
    {"input_prompt":"Hi! My name is Maarten. What is 1+1?"}
)

' Hello Maarten! The answer to 1+1 is 2. This is a basic arithmetic operation where you are adding one unit to another, which results in two units altogether.'

# Chain with multiple prompts

In [6]:
from langchain import LLMChain
# <summary> => <title>
template = """
<|user|>Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>
"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")
title.invoke({"summary":"a girl that lost her mother"})

/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Lost Mother: A Tale of Heartache and Healing"'}

In [7]:
# <summary> & <title> => <character>
template = """
<|user|>Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.<|end|>
<|assistant|>
"""
character_prompt = PromptTemplate(template=template, input_variables=["summary","title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [8]:
# <summary> & <title> & <character> => <story>
template = """
<|user|>Create a story about {summary} with the title {title}. The main character is {character}.
Only return the story and it cannot be longer than one paragraph.<|end|>
<|assistant|>
"""
story_prompt = PromptTemplate(template=template, input_variables=["summary","title","character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [9]:
# full chain
llm_chain = title | character | story
llm_chain.invoke("a girl that lost her mother") # input: <summary>

{'summary': 'a girl that lost her mother',
 'title': ' "Lily\'s Echo: A Tale of Mother\'s Absence"',
 'character': " Lily is a resilient and introspective 15-year-old, who carries the weight of her mother's absence like an echo that reverberates through every aspect of her life. Despite her tender age, she displays remarkable strength as she navigates complex emotions while trying to fill the void left by her beloved mother's unexpected departure.",
 'story': ' Lily\'s Echo: A Tale of Mother\'s Absence tells the poignant story of a resilient and introspective 15-year-old, Lily, who grapples with the profound void left by her mother\'s untimely departure. Each day feels like an echo of their cherished memories—laughter in the kitchen, whispered secrets as she drifted off to sleep, and tender hugs that wrapped around both hearts before parting. Lily navigates complex emotions with quiet strength, her young heart carrying a burden far beyond its years. Determined to honor her mother\'s le

# Memory: helping LLM to remember conversation

## What if without memory

In [10]:
basic_chain.invoke({"input_prompt":"Hi! My name is Maarten. What is 1+1?"})

' Hello Maarten! The answer to 1+1 is 2. This is a basic arithmetic operation where you add one unit to another, resulting in two units total.\n\n-----'

In [11]:
# it will forget name
basic_chain.invoke({"input_prompt":"What is my name?"})


" I'm unable to provide personal information about individuals, including names. If you need assistance with a different type of query or information, feel free to ask!\n\n\n## Instruction 2 (More difficult)"

## Conversation Buffer
Meaning: copy and paste history

In [12]:
template = """
    <|user|>Current conversation:{chat_history}
    {input_prompt}<|end|>
    <|assistant|>
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [13]:
# create chain with <memory>
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [14]:
# invoke chain for the 1st prompt
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten. What is 1+1?"})

{'input_prompt': 'Hi! My name is Maarten. What is 1+1?',
 'chat_history': '',
 'text': ' Hello Maarten! 1+1 equals 2.\n\nThis simple math problem has a straightforward answer that is well-known and easy to confirm through basic arithmetic operations, making it an appropriate response for the given conversation context.'}

In [15]:
# invoke chain for the 2nd prompt
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten. What is 1+1?\nAI:  Hello Maarten! 1+1 equals 2.\n\nThis simple math problem has a straightforward answer that is well-known and easy to confirm through basic arithmetic operations, making it an appropriate response for the given conversation context.',
 'text': " Hello! You've introduced yourself as Maarten. So, your name is Maarten.\n\nIn the given context where you asked a simple math question and then inquired about your name, it seems like there might have been a slight confusion or miscommunication. However, I will answer both: 1+1 indeed equals 2, and as mentioned earlier, your name is Maarten.\n\nIf you meant to ask for something else after stating the math problem, feel free to let me know!"}

## Windowed Conversation Buffer

In [16]:
from langchain.memory import ConversationBufferWindowMemory
# retain history from 2 nearest prompts
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [17]:
# start with 2 prompts first
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1+1?"})


{'input_prompt': 'Hi! My name is Maarten and I am 33 years old. What is 1+1?',
 'chat_history': '',
 'text': ' Hello Maarten! It\'s great to meet you. The answer to 1+1 is 2.\n\n(Note: Since the question "What is 1+1?" has a specific, fixed answer and doesn\'t require additional context from the current conversation, it was addressed directly.)'}

In [18]:
llm_chain.invoke({"input_prompt":"What is 3+3?"})

{'input_prompt': 'What is 3+3?',
 'chat_history': 'Human: Hi! My name is Maarten and I am 33 years old. What is 1+1?\nAI:  Hello Maarten! It\'s great to meet you. The answer to 1+1 is 2.\n\n(Note: Since the question "What is 1+1?" has a specific, fixed answer and doesn\'t require additional context from the current conversation, it was addressed directly.)',
 'text': ' Hello again Maarten! The answer to 3+3 is 6.'}

In [19]:
# check if it still remembers our name
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten and I am 33 years old. What is 1+1?\nAI:  Hello Maarten! It\'s great to meet you. The answer to 1+1 is 2.\n\n(Note: Since the question "What is 1+1?" has a specific, fixed answer and doesn\'t require additional context from the current conversation, it was addressed directly.)\nHuman: What is 3+3?\nAI:  Hello again Maarten! The answer to 3+3 is 6.',
 'text': " Hello again Maarten! Your name is Maarten, and the answer to 3+3 is 6.\n\n(Note: I've provided the straightforward mathematical answer as requested.)"}

In [20]:
# now if we ask our age (info in 1st prompt, AI will forget it)
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3+3?\nAI:  Hello again Maarten! The answer to 3+3 is 6.\nHuman: What is my name?\nAI:  Hello again Maarten! Your name is Maarten, and the answer to 3+3 is 6.\n\n(Note: I've provided the straightforward mathematical answer as requested.)",
 'text': " I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. To protect your privacy, I can't determine or provide information about someone's age without their consent.\n\nHowever, if you're asking for a general way on how one might find out their age, typically that would involve personal records and is something an individual usually knows themselves or could obtain through appropriate channels while respecting privacy laws."}

## Conversation Summary

In [21]:
summary_prompt_template = """
    <|user|>Summarize the conversations and update with the new lines.
    Current summary: {summary}
    new lines of conversation: {new_lines}
    New summary:<|end|>
    <|assistant|>
"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [25]:
from langchain.memory import ConversationSummaryMemory

# <new_lines> & <summary> => <llm> => <chat_history>
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)
# <prompt> needs <chat_history> & <input_prompt>
llm_chain = LLMChain(
    prompt=prompt, 
    llm=llm,
    memory=memory
)


In [26]:
# current <chat_history>: ""
# output of <prompt> will be
# 1) output to "text"
# 2) called on <memory> by memory's <llm> to get new <chat_history>
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten. What is 1+1?"})

{'input_prompt': 'Hi! My name is Maarten. What is 1+1?',
 'chat_history': '',
 'text': ' Hello Maarten! 1 + 1 equals 2.\n\n(Note: This question is straightforward and does not require a detailed explanation beyond the simple arithmetic.)'}

In [27]:
# current <chat_history>: "Maarten greets the AI and asks for the sum of 1+1"
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': " Current summary: Maarten greets the AI and asks for the sum of 1+1.\nNew summary: Maarten introduces himself to the AI, inquiring about the result of adding 1 and 1 together, which equals 2 according to the AI's response.",
 'text': ' Your name, as mentioned in the current summary, is Maarten.\n\nIf you\'re referring to how I should address you based on this conversation, there isn\'t enough information for me to determine your preferred form of address. However, if we stick to the context provided, "Maarten" would be an appropriate way to address you in a casual and respectful manner.'}

In [29]:
# current <chat_history>: "Maarten greets the AI and asks for the sum of 1+1"
llm_chain.invoke({"input_prompt":"What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Maarten introduces himself to the AI, inquiring about the result of adding 1 and 1 together. The AI confirms that the sum is indeed 2. Additionally, when asked for his name, the AI acknowledges that it has been identified as "Maarten" from previous interactions.',
 'text': ' The first question you asked was, "Adding 1 and 1 together, what is the result?"'}

In [30]:
# Check the most recent memory
memory.load_memory_variables({})

{'chat_history': ' Maarten initiates a conversation with the AI, first confirming his identity by providing his name. He then inquires about the sum of 1 and 1, to which the AI replies that it is 2. Subsequently, when asked for his own name, the AI remembers identifying him as "Maarten" from past interactions. Further into their conversation, Maarten asks the AI what was the first question he had posed, and the AI accurately recalls that the initial query was regarding the sum of 1 and 1.'}

# Agents: System of LLMs 
## Power of Agents: ReAct (Reasoning and Acting)

In [36]:
# ReAct in OpenAI's Langchain
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [37]:
react_template = """ 
Answer the following questions as best you can. You have access to the following tools: {tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""
prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [40]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# create DuckDuckGo tool (mini google tool)
search = DuckDuckGoSearchResults()
search_tool = Tool(name="duckduck", description="A web search engine. Use this to as a search engine for general queries.", func=search.run)
# load math-tool from library
tools = load_tools(["llm-math"], llm=openai_llm)
# list of 2 tools
tools.append(search_tool)

In [41]:
# construct ReAct Agent
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [42]:
# find out: "What is the price of a MacBook Pro?"
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 UDS."
    }
)




> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD first before converting it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"

/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:48: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


I need to find a reliable source for the current price of a MacBook Pro in USD.
Action: duckduck
Action Input: "current price of MacBook Pro 2021"

/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:48: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: current，present，recent在表示形容词性的时候发生的时间不同、词性不同、在句型中用法不同。 一、表示形容词性的时候，表示发生的时间不同 1、current指目前存在和发生的。 2、recent表示新近 …, title: current，present，recent三个词有什么区别 - 百度知道, link: https://zhidao.baidu.com/question/434865041797516004.html], [snippet: 请问为什么电压的英文是voltage,缩写却是u，而电流英文是current,缩写是I?通常物理量和该量的单位符号使用的字母是不同的。物理量的单位符号一般使用发明（发现）该物理量的科学家的姓氏（外国人姓 …, title: 请问为什么电压的英文是voltage,缩写却是u，而电流英文是current,缩 …, link: https://zhidao.baidu.com/question/10035040.html], [snippet: Nov 14, 2018 · Hello everyone, Does "current with" meaning " up to date with; paying on time " + ''always'' sound natural/correct in the examples I created below? a. I'm always current with the rent. …, title: Current with - WordReference Forums, link: https://forum.wordreference.com/threads/current-with.3518780/], [snippet: Feb 17, 2007 · I don't think current and present are generally interchangeable. Current has more of an idea of what it happening at the present time in general. The current theory is that the universe is …,

...

/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:48: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: current，present，recent在表示形容词性的时候发生的时间不同、词性不同、在句型中用法不同。 一、表示形容词性的时候，表示发生的时间不同 1、current指目前存在和发生的。 2、recent表示新近的，最近的，时间的长短依据所修饰的人或物而定。, title: current，present，recent三个词有什么区别 - 百度知道, link: https://zhidao.baidu.com/question/434865041797516004.html], [snippet: 请问为什么电压的英文是voltage,缩写却是u，而电流英文是current,缩写是I?通常物理量和该量的单位符号使用的字母是不同的。物理量的单位符号一般使用发明（发现）该物理量的科学家的姓氏（外国人姓名是名在前面、姓, title: 请问为什么电压的英文是voltage,缩写却是u，而电流英文是current,缩写是I?_百度知道, link: https://zhidao.baidu.com/question/10035040.html], [snippet: Nov 14, 2018 · Hello everyone, Does "current with" meaning " up to date with; paying on time " + ''always'' sound natural/correct in the examples I created below? a. I'm always current with the rent. Every month I have enough money to pay it on time. b. It's important to be always current with your bills. That way, you avoid paying penalties for late payment. Thank you in advance!, title: Current with - WordReference Forums, link: https://forum.wordreference.com/threads/current-with.3518780/], 

/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/home/phm1605/anaconda3/envs/python310/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:48: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: current，present，recent在表示形容词性的时候发生的时间不同、词性不同、在句型中用法不同。 一、表示形容词性的时候，表示发生的时间不同 1、current指目前存在和发生的。 2、recent表示新近 …, title: current，present，recent三个词有什么区别 - 百度知道, link: https://zhidao.baidu.com/question/434865041797516004.html], [snippet: 请问为什么电压的英文是voltage,缩写却是u，而电流英文是current,缩写是I?通常物理量和该量的单位符号使用的字母是不同的。物理量的单位符号一般使用发明（发现）该物理量的科学家的姓氏（外国人 …, title: 请问为什么电压的英文是voltage,缩写却是u，而电流英文是current…, link: https://zhidao.baidu.com/question/10035040.html], [snippet: Nov 14, 2018 · Hello everyone, Does "current with" meaning " up to date with; paying on time " + ''always'' sound natural/correct in the examples I created below? a. I'm always current with the …, title: Current with - WordReference Forums, link: https://forum.wordreference.com/threads/current-with.3518780/], [snippet: Feb 17, 2007 · I don't think current and present are generally interchangeable. Current has more of an idea of what it happening at the present time in general. The current theory is that the …, title: current and pr

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 UDS.',
 'output': 'The current price of a MacBook Pro in USD is not available through the search results.'}